In [1]:
%matplotlib inline
import os
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
import torch
import sys
from shapely.geometry import mapping
sys.path.append(r"E:\gitlab\geoloc2\Detecciondeterrenos")
from codigos import Generar_txt
###path de yolo dentro de computadora
os.chdir(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7")
from detect_Alberto_v4 import *

In [2]:
Modelo=modelo(weights="Modelos/best.pt")#,device="cpu") ## device="0" para gpu

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 



## Parametros del raster

In [3]:
raster=r"C:\Users\ASUS\Desktop\neza1.tif"

In [4]:
gdal_interpeter = gdal.Open(raster)
width = gdal_interpeter.RasterXSize
height = gdal_interpeter.RasterYSize
coordenadas_gdal = gdal_interpeter.GetGeoTransform()
minx = coordenadas_gdal[0]
miny = coordenadas_gdal[3] + width*coordenadas_gdal[4] + height*coordenadas_gdal[5] 
maxx = coordenadas_gdal[0] + width*coordenadas_gdal[1] + height*coordenadas_gdal[2]
maxy = coordenadas_gdal[3]
src_raster_path = raster
src=rasterio.open(src_raster_path)
H,W=src.shape
src.crs,H,W,minx,maxx,miny,maxy,"W_metros",maxx-minx,"H_metros",maxy-miny

(CRS.from_epsg(3857),
 155650,
 112773,
 -11028184.223,
 -11016906.9436,
 2198108.5665,
 2213673.5468,
 'W_metros',
 11277.279399998486,
 'H_metros',
 15564.980299999937)

In [5]:
dim=int(np.ceil(map_d(minx+120,minx,maxx,0,W)))
alto=int(np.floor(H/dim))
ancho=int(np.floor(W/dim))
alto,ancho,alto*ancho
nombres=[]
imshow=False
result=pd.DataFrame()
casas=[]
terreno=[]
angulosget=[]
conf_casas=[]
conf_terreno=[]
alto,ancho,alto*ancho,dim

(129, 93, 11997, 1201)

## Generar prediccion sobre raster(municipio) con salida en shp

In [6]:
## shape igecem de manzanas por municipio (solo pora tener la delimitacion geografica del municipio)
shape=gpd.read_file(r"C:\Users\ASUS\Documents\GobiernoEdomex\Agua\geoshapes\Full manzanas\Manzana_Nezahualcoyotl.shp")
shape=shape.to_crs("3857")
c=[]
for manzana in range(len(shape)):
    proyecciones1=mapping(shape["geometry"][manzana]).get("coordinates")
    if len(proyecciones1)>1:
        for manzan in proyecciones1:
            print(manzana)
            try:
                proyecciones=manzan
            except:
                proyecciones=manzan[0]
            point1=np.min((proyecciones,proyecciones),axis=1)[0]
            min_y,min_x=point1[0],point1[1]
            point2=np.max((proyecciones,proyecciones),axis=1)[0]
            max_y,max_x=point2[0],point2[1]
            c.append(",".join([str(min_y),str(min_x),str(max_y),str(max_x)]))#,proyecciones
    else:
        proyecciones=proyecciones1[0]
        point1=np.min((proyecciones,proyecciones),axis=1)[0]
        min_y,min_x=point1[0],point1[1]
        point2=np.max((proyecciones,proyecciones),axis=1)[0]
        max_y,max_x=point2[0],point2[1]
        c.append(",".join([str(min_y),str(min_x),str(max_y),str(max_x)]))#,proyecciones    
    
    # point1=np.min((proyecciones,proyecciones),axis=1)[0]
    # min_y,min_x=point1[0],point1[1]
    # point2=np.max((proyecciones,proyecciones),axis=1)[0]
    # max_y,max_x=point2[0],point2[1]
    # c.append(",".join([str(min_y),str(min_x),str(max_y),str(max_x)]))#,proyecciones
shape1=pd.DataFrame()
shape1["points"]=c
shape1=shape1["points"].str.split(",",expand=True)
shape1=shape1.astype({0:"float64",1:"float64",2:"float64",3:"float64"})
shape=shape1

In [7]:
imshow=False

with tqdm.tqdm(total=alto*ancho) as pbar:
    for j in range(ancho):#ancho
        for i in (range(alto)):#alto
            k = cv2.waitKey(1)
            if k == ord('s'):
                print("entre aqui")
            generar=0
            label=raster.replace("\\","/").split("/")[-1][:-4]+"_"
            nameimg=label.lower()+str(i)+"_"+str(j)
            nombres.append(nameimg)
            cuadro=[]
            for k in range(2):
                for l in range(2):
                    cuadro.append((minx+(maxx-minx)/ancho*(j+k),
                                maxy-(maxy-miny)/alto*(i+l),
                                0.0))
            cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
            for punto in cuadro:
                x=float(punto[0])
                y=float(punto[1])
                if len(shape[(shape[0]<=x)&(shape[2]>=x)&(shape[1]<=y)&(shape[3]>=y)])>0:
                    generar=1             
            if generar==1:
                shapes=[{"type":'Polygon','coordinates':[cuadro]}]
                vector=[]
                array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
                four_images=[array[2],array[1],array[0]]
                imagen_n = np.stack(four_images, axis=-1)
                angulo_f=0
                anterior=0
                actual=1
                vector1=[]
                angulo=0
                le=0
                angulo_1=correct_orientation(imagen_n,dim)[0]
                for i in range(-2,2,1):
                    angulo=angulo_1+i
                    anterior=len(vector)
                    image_ro=imagen_n.copy()
                    M = cv2.getRotationMatrix2D((dim//2,dim//2), angulo, 1)
                    image_ro = cv2.warpAffine(image_ro, M, (dim,dim))
                    with torch.no_grad():
                        vector1=Modelo.detect(opt_source="cache1.png",opt_conf_thres=0.01,imagen_s=image_ro)
                    actual=len(vector)
                    if len(vector1)>le:
                        le=len(vector1)
                        vector=vector1
                        imagen_n1=image_ro
                        angulo_f=angulo        
                vector2=Modelo.detect(opt_source="cache1.png",opt_conf_thres=0.01,imagen_s=imagen_n)
                if len(vector)>len(vector2):
                    imagen_n1=image_ro
                    angulo_f=angulo
                else:
                    le=len(vector2)
                    vector=vector2
                    imagen_n1=imagen_n
                    angulo_f=0
                angulo=angulo_f
    #                 if angulo!=0:
    #                     imshow=True
                angulosget.append(angulo)
                proyecciones=shapes[0].get("coordinates")[0][:-1]
                df_cache=vector2xy(vector,dim=dim,nameimg=nameimg)
                for i in (range(len(df_cache))):
                    if df_cache["Tipo"][i]=="casa":
                        casas.append(rotacion_detect(df_cache.loc[i,"start_point_100"], df_cache.loc[i,"end_point_100"],-angulo,proyecciones=proyecciones))
                        conf_casas.append(df_cache.loc[i,"conf"])
                    else:
                        terreno.append(rotacion_detect(df_cache.loc[i,"start_point_100"], df_cache.loc[i,"end_point_100"],-angulo,proyecciones=proyecciones))
                        conf_terreno.append(df_cache.loc[i,"conf"])
                if imshow:
                    print(angulo)
                    imshow_detect(df_cache,imagen_n1)
                    #imshow=False

            pbar.update(1)
            


 23%|██▎       | 2772/11997 [41:18<6:25:13,  2.51s/it]  

casas

In [ ]:
gdf_casas=gpd.GeoDataFrame({"conf":conf_casas},geometry=casas,crs=src.crs)
gdf_casas.set_crs=src.crs
gdf_casas["area"]=gdf_casas.area
gdf_casas=gdf_casas.astype({"conf":"float64"})
# gdf_casas=gdf_casas[(gdf_casas["conf"]<)&(gdf_casas["conf"]>.09)]
gdf_casas.to_file(r"C:/Users/ASUS/salida/shape/neza_new_casas.shp")
gdf_terreno=gpd.GeoDataFrame({"conf":conf_terreno},geometry=terreno,crs=src.crs)
gdf_terreno.set_crs=src.crs
gdf_terreno.to_file(r"C:/Users/ASUS/salida/shape/neza_new_terreno.shp")

## Generar imagenes mosaico

In [ ]:
# Path donde se guardan images
path_mosaico_salida=r"D:\instream\Mi unidad\geoshaps\neza\images/"
generar_imagenes=int(input("Generar imagenes: 1 si, 0 no "))
if generar_imagenes==1:
    generar_imagenes_sinrotar=int(input("Generar imagenes sin rotar: 1 si, 0 no "))
    generar_imagenes_rotadas=int(input("Generar imagenes rotadas: 1 si, 0 no "))
    with tqdm.tqdm(total=alto*ancho) as pbar:
        for j in range(0,ancho):#ancho
            for i in (range(alto)):#alto
                generar=0
                label=raster.replace("\\","/").split("/")[-1][:-4]+"_"
                nameimg=label.lower()+str(i)+"_"+str(j)
                cuadro=[]
                for k in range(2):
                    for l in range(2):
                        cuadro.append((minx+(maxx-minx)/ancho*(j+k),
                                       maxy-(maxy-miny)/alto*(i+l),
                                       0.0))
                cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
                for punto in cuadro:
                    x=float(punto[0])
                    y=float(punto[1])
                    if len(shape[(shape[0]<=x)&(shape[2]>=x)&(shape[1]<=y)&(shape[3]>=y)])>0:
                        generar=1             
                if generar==1:
                    shapes=[{"type":'Polygon','coordinates':[cuadro]}]
                    array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
                    four_images=[array[2],array[1],array[0]]
                    imagen_n = np.stack(four_images, axis=-1)
                    if generar_imagenes_sinrotar==1:
                        cv2.imwrite(path_mosaico_salida+nameimg+".png",imagen_n)
                    if generar_imagenes_rotadas==1:
                        angulo_1,imagen_ro=correct_orientation(imagen_n,dim)
                        cv2.imwrite(path_mosaico_salida+nameimg+"_"+str(angulo_1)+".png",imagen_ro)
                pbar.update(1)

## Generar prediccion sobre imagen con salida txt

In [ ]:
# path_mosaico_salida="D:/tlalne/"
vector=Modelo.detect(opt_source=path_mosaico_salida,opt_conf_thres=0.08)
Generar_txt(vector,path_mosaico_salida+"labels")
with open(path_mosaico_salida+"labels/classes.txt","w") as f:
    f.writelines("\n".join(["casa","terreno_baldio"]))